In [1]:
import pandas as pd
import numpy as np
import os
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle 
pd.set_option('display.max_columns', None)

In [2]:
big_df = pd.read_csv("../data/big_df.csv")
big_df

,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_reach_dif,blue_height_dif,winner,weight_class,title_fight
0,0,31.0,36.0,76,75,Ciryl Gane,Derrick Lewis,81,79,Orthodox,Orthodox,512,570,904,1087,470,519,861,1030,5,7,23,26,2,0,483,1428,2,7,0,3,99,71,133,97,173,440,503,902,89,25,108,34,321,199,684,553,36,81,47,125,4,256,13,355,256,867,667,1272,238,433,646,787,0,35,3,77,0,7,3,3858,0,2,0,3,99,71,133,97,173,440,50

In [3]:
future_df = big_df[big_df["winner"] == "tbd"]
future_df

,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_reach_dif,blue_height_dif,winner,weight_class,title_fight
0,0,31.0,36.0,76,75,Ciryl Gane,Derrick Lewis,81,79,Orthodox,Orthodox,512,570,904,1087,470,519,861,1030,5,7,23,26,2,0,483,1428,2,7,0,3,99,71,133,97,173,440,503,902,89,25,108,34,321,199,684,553,36,81,47,125,4,256,13,355,256,867,667,1272,238,433,646,787,0,35,3,77,0,7,3,3858,0,2,0,3,99,71,133,97,173,440,50

In [4]:
number = LabelEncoder()
# data['red_name'] = number.fit_transform(data['red_name'].astype('str'))
# data['blue_name'] = number.fit_transform(data['blue_name'].astype('str'))
future_df['red_stance'] = number.fit_transform(future_df['red_stance'].astype('str'))
future_df['blue_stance'] = number.fit_transform(future_df['blue_stance'].astype('str'))
future_df['weight_class'] = number.fit_transform(future_df['weight_class'].astype('str'))
future_df

<ipython-input-4-0918a8733ed1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df['red_stance'] = number.fit_transform(future_df['red_stance'].astype('str'))
<ipython-input-4-0918a8733ed1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df['blue_stance'] = number.fit_transform(future_df['blue_stance'].astype('str'))
<ipython-input-4-0918a8733ed1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_reach_dif,blue_height_dif,winner,weight_class,title_fight
0,0,31.0,36.0,76,75,Ciryl Gane,Derrick Lewis,81,79,0,0,512,570,904,1087,470,519,861,1030,5,7,23,26,2,0,483,1428,2,7,0,3,99,71,133,97,173,440,503,902,89,25,108,34,321,199,684,553,36,81,47,125,4,256,13,355,256,867,667,1272,238,433,646,787,0,35,3,77,0,7,3,3858,0,2,0,3,99,71,133,97,173,440,503,902,89,25,10

In [5]:
data = future_df.drop(columns=["lev1", "winner", "red_name", "blue_name"])
# target = train_test_df["winner"]
# feature_names = data.columns

In [6]:
loaded_model = pickle.load(open("../saved_models/rf_up_model.sav", 'rb'))
pred = loaded_model.predict(data)
future_df["prediction"] = pred
future_df

/home/audricperger/anaconda3/envs/pyenv/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/audricperger/anaconda3/envs/pyenv/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
<ipython-input-6-80f10447a0f1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df["prediction"] = pred


,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_reach_dif,blue_height_dif,winner,weight_class,title_fight,prediction
0,0,31.0,36.0,76,75,Ciryl Gane,Derrick Lewis,81,79,0,0,512,570,904,1087,470,519,861,1030,5,7,23,26,2,0,483,1428,2,7,0,3,99,71,133,97,173,440,503,902,89,25,108,34,321,199,684,553,36,81,47,125,4,256,13,355,256,867,667,1272,238,433,646,787,0,35,3,77,0,7,3,3858,0,2,0,3,99,71,133,97,173,440,503,9

In [7]:
pcts = loaded_model.predict_proba(data)
favs = []
dogs = []
for pct in pcts:
    fav = max(pct)
    dog = min(pct)
    favs.append(fav)
    dogs.append(dog)

In [8]:
future_df["fav_pct"] = favs
future_df["dog_pct"] = dogs
future_df

<ipython-input-8-a2d709097410>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df["fav_pct"] = favs
<ipython-input-8-a2d709097410>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df["dog_pct"] = dogs


,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_reach_dif,blue_height_dif,winner,weight_class,title_fight,prediction,fav_pct,dog_pct
0,0,31.0,36.0,76,75,Ciryl Gane,Derrick Lewis,81,79,0,0,512,570,904,1087,470,519,861,1030,5,7,23,26,2,0,483,1428,2,7,0,3,99,71,133,97,173,440,503,902,89,25,108,34,321,199,684,553,36,81,47,125,4,256,13,355,256,867,667,1272,238,433,646,787,0,35,3,77,0,7,3,3858,0,2,0,3,99,71,133,

In [9]:
future_df["fav_pct"] = future_df["fav_pct"].round(2)
future_df["dog_pct"] = future_df["dog_pct"].round(2)
future_df

<ipython-input-9-dd52a5942276>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df["fav_pct"] = future_df["fav_pct"].round(2)
<ipython-input-9-dd52a5942276>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df["dog_pct"] = future_df["dog_pct"].round(2)


,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_reach_dif,blue_height_dif,winner,weight_class,title_fight,prediction,fav_pct,dog_pct
0,0,31.0,36.0,76,75,Ciryl Gane,Derrick Lewis,81,79,0,0,512,570,904,1087,470,519,861,1030,5,7,23,26,2,0,483,1428,2,7,0,3,99,71,133,97,173,440,503,902,89,25,108,34,321,199,684,553,36,81,47,125,4,256,13,355,256,867,667,1272,238,433,646,787,0,35,3,77,0,7,3,3858,0,2,0,3,99,71,133,

In [10]:
future_df["fav_moneyline"] = (future_df["fav_pct"] * 100) / (100 - (future_df["fav_pct"] * 100)) * (-100)
future_df["dog_moneyline"] = ((100 - (future_df["dog_pct"] * 100)) / (future_df["dog_pct"] * 100) * 100)
future_df

<ipython-input-10-e94fd81e017f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df["fav_moneyline"] = (future_df["fav_pct"] * 100) / (100 - (future_df["fav_pct"] * 100)) * (-100)
<ipython-input-10-e94fd81e017f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df["dog_moneyline"] = ((100 - (future_df["dog_pct"] * 100)) / (future_df["dog_pct"] * 100) * 100)


,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_reach_dif,blue_height_dif,winner,weight_class,title_fight,prediction,fav_pct,dog_pct,fav_moneyline,dog_moneyline
0,0,31.0,36.0,76,75,Ciryl Gane,Derrick Lewis,81,79,0,0,512,570,904,1087,470,519,861,1030,5,7,23,26,2,0,483,1428,2,7,0,3,99,71,133,97,173,440,503,902,89,25,108,34,321,199,684,553,36,81,47,125,4,256,13,355,256,867,667,1272,238,433,646,787,0,35,3,77,0

In [11]:
future_df["fav_moneyline"] = future_df["fav_moneyline"].astype(int)
future_df["dog_moneyline"] = future_df["dog_moneyline"].astype(int)
future_df

<ipython-input-11-cdffb34718b9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df["fav_moneyline"] = future_df["fav_moneyline"].astype(int)
<ipython-input-11-cdffb34718b9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df["dog_moneyline"] = future_df["dog_moneyline"].astype(int)


,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_reach_dif,blue_height_dif,winner,weight_class,title_fight,prediction,fav_pct,dog_pct,fav_moneyline,dog_moneyline
0,0,31.0,36.0,76,75,Ciryl Gane,Derrick Lewis,81,79,0,0,512,570,904,1087,470,519,861,1030,5,7,23,26,2,0,483,1428,2,7,0,3,99,71,133,97,173,440,503,902,89,25,108,34,321,199,684,553,36,81,47,125,4,256,13,355,256,867,667,1272,238,433,646,787,0,35,3,77,0

In [12]:
my_picks = pd.read_csv("picks/rolling_picks.csv")
my_picks = my_picks.drop(columns="Unnamed: 0")
my_picks

,blue_name,red_name,prediction,fav_pct,dog_pct,fav_moneyline,dog_moneyline
0,Ciryl Gane,Derrick Lewis,red,0.52,0.48,-108,108
1,Pedro Munhoz,Jose Aldo,red,0.58,0.42,-138,138
2,Vicente Luque,Michael Chiesa,red,0.58,0.42,-138,138
3,Angela Hill,Tecia Torres,red,0.71,0.29,-244,244
4,Casey Kenney,Song Yadong,red,0.63,0.37,-170,170
5,Rafael Fiziev,Bobby Green,blue,0.62,0.38,-163,163
6,Drako Rodriguez,Vince Morales,red,0.63,0.37,-170,170
7,Ed Herman,Alonzo Menifield,blue,0.50,0.50,-100,100
8,Jessica Penne,Karolina Kowalkiewicz,blue,0.54,0.46,-117,117
9,Ode Osbourne,Manel Kape,red,0.58,0.42,-138,138


In [13]:
future_picks = future_df[["blue_name", "red_name", "prediction", "fav_pct", "dog_pct", \ 
                          "fav_moneyline", "dog_moneyline"]]
future_picks

,blue_name,red_name,prediction,fav_pct,dog_pct,fav_moneyline,dog_moneyline
0,Ciryl Gane,Derrick Lewis,red,0.52,0.48,-108,108
80,Pedro Munhoz,Jose Aldo,red,0.58,0.42,-138,138
173,Vicente Luque,Michael Chiesa,red,0.58,0.42,-138,138
239,Angela Hill,Tecia Torres,red,0.71,0.29,-244,244
279,Casey Kenney,Song Yadong,red,0.63,0.37,-170,170
292,Rafael Fiziev,Bobby Green,blue,0.62,0.38,-163,163
341,Drako Rodriguez,Vince Morales,red,0.63,0.37,-170,170
343,Ed Herman,Alonzo Menifield,blue,0.50,0.50,-100,100
372,Jessica Penne,Karolina Kowalkiewicz,blue,0.54,0.46,-117,117
393,Ode Osbourne,Manel Kape,red,0.58,0.42,-138,138


In [14]:
my_picks = pd.concat([future_picks, my_picks])
my_picks.to_csv("picks/rolling_picks.csv")
my_picks

,blue_name,red_name,prediction,fav_pct,dog_pct,fav_moneyline,dog_moneyline
0,Ciryl Gane,Derrick Lewis,red,0.52,0.48,-108,108
1,Pedro Munhoz,Jose Aldo,red,0.58,0.42,-138,138
2,Vicente Luque,Michael Chiesa,red,0.58,0.42,-138,138
3,Angela Hill,Tecia Torres,red,0.71,0.29,-244,244
4,Casey Kenney,Song Yadong,red,0.63,0.37,-170,170
5,Rafael Fiziev,Bobby Green,blue,0.62,0.38,-163,163
6,Drako Rodriguez,Vince Morales,red,0.63,0.37,-170,170
7,Ed Herman,Alonzo Menifield,blue,0.50,0.50,-100,100
8,Jessica Penne,Karolina Kowalkiewicz,blue,0.54,0.46,-117,117
9,Ode Osbourne,Manel Kape,red,0.58,0.42,-138,138
